In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 87.7 MB/s eta 0:00:00


In [19]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import transformers

print(transformers.__version__)

4.26.1


## Preparing the dataset

In [4]:
from datasets import load_dataset
datasets = load_dataset('Sree1994/babylm_childstories')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/1200 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/4800 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Sree1994___parquet/Sree1994--babylm_childstories-3b1a91b15b07a2d3/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
datasets["train"][10]

{'text': "Gangazara sat in the dark cellar, without the sun's light falling upon him, and feasted upon the breadcrumbs and sweetmeats that the rats so kindly supplied him with. These delicacies had completely changed his body into a red, stout, huge, unwieldy mass of flesh. Thus passed full ten years, as prophesied in the horoscope.\n"}

## Causal Language modeling

In [6]:
model_checkpoint = "EleutherAI/gpt-neo-1.3B"

In [7]:
from transformers import AutoTokenizer
import torch

    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [9]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/4800 [00:00<?, ? examples/s]

In [10]:
tokenized_datasets["train"][1]

{'input_ids': [1,
  4366,
  4692,
  2330,
  1243,
  553,
  484,
  531,
  11,
  366,
  66,
  480,
  256,
  14739,
  866,
  2402,
  514,
  290,
  12008,
  514,
  13,
  383,
  2330,
  1243,
  1422,
  470,
  910,
  257,
  1573,
  11,
  475,
  484,
  1625,
  523,
  3049,
  326,
  356,
  892,
  484,
  1276,
  307,
  6776,
  13,
  14026,
  514,
  644,
  284,
  466,
  13,
  12039,
  356,
  7808,
  1701,
  198],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [11]:
# block_size = tokenizer.model_max_length
block_size = 128

In [12]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [13]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/1200 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4800 [00:00<?, ? examples/s]

In [14]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' we should have been married then if you had not gone away so suddenly, and for long, long years I followed you." "Well," said Majnun, "if you can make yourself a young girl again, I will marry you."\nThe custom yet noticed sometimes in country churches and festive gatherings of placing the males and females on opposite sides of the room was originated not so much as a punishment to both, as to give the men an opportunity to act together when the red brother felt ill at ease.\nAugust 5, Commodore Farragut captured Mobile, after a neat and attractive naval fight, and on the 24th'

In [15]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [16]:
from transformers import Trainer, TrainingArguments

We pass along all of those to the `Trainer` class:

In [17]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-babylm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)

Cloning https://huggingface.co/vj99/gpt-neo-1.3B-finetuned-babylm into local empty directory.


And we can train our model:

In [21]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2737
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1029
  Number of trainable parameters = 1315575808


Epoch,Training Loss,Validation Loss
1,No log,2.767713
2,2.685400,2.797864
3,2.034800,2.938869


***** Running Evaluation *****
  Num examples = 697
  Batch size = 8
Saving model checkpoint to gpt-neo-1.3B-finetuned-babylm/checkpoint-500
Configuration saved in gpt-neo-1.3B-finetuned-babylm/checkpoint-500/config.json
Configuration saved in gpt-neo-1.3B-finetuned-babylm/checkpoint-500/generation_config.json
Model weights saved in gpt-neo-1.3B-finetuned-babylm/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 697
  Batch size = 8
Saving model checkpoint to gpt-neo-1.3B-finetuned-babylm/checkpoint-1000
Configuration saved in gpt-neo-1.3B-finetuned-babylm/checkpoint-1000/config.json
Configuration saved in gpt-neo-1.3B-finetuned-babylm/checkpoint-1000/generation_config.json
Model weights saved in gpt-neo-1.3B-finetuned-babylm/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 697
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1029, training_loss=2.347645906943274, metrics={'train_runtime': 1031.5764, 'train_samples_per_second': 7.96, 'train_steps_per_second': 0.998, 'total_flos': 7620576295256064.0, 'train_loss': 2.347645906943274, 'epoch': 3.0})

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [22]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 697
  Batch size = 8


Perplexity: 18.89
